In [ ]:
from DeepFMKit.core import DeepFitFramework
from DeepFMKit.plotting import default_rc
from DeepFMKit.experiments import Experiment
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update(default_rc)

In [ ]:
dff = DeepFitFramework(raw_file='../test/raw_data.txt', raw_labels=['raw1'])
dff.fit('raw1', n=20)

In [ ]:
axes = dff.plot()
plt.show()

In [ ]:
ax = dff.raws['raw1'].plot(xrange=[0,0.01])
plt.show()

In [ ]:
"""Defines, runs, and plots a 2D experiment with the final runner API."""
exp2d = Experiment(description="2D Sweep of Modulation and IFO Phase")

# Define the two axes to sweep
exp2d.add_axis('psi', np.linspace(-np.pi, np.pi, 25))
exp2d.add_axis('ifo_phi', np.linspace(-np.pi, np.pi, 25))

# Define fixed physical parameters
exp2d.set_static({'m_target': 6.0, 'opd': 0.2})

# Map the experimental variables to their corresponding object attributes
exp2d.map_parameter(name='psi', target_object='laser_config', target_attribute='psi')
exp2d.map_parameter(name='ifo_phi', target_object='main_ifo_config', target_attribute='phi')

# Define the analysis to run. All standard fit parameters will be extracted.
exp2d.add_analysis(
    name='nls_fit',
    fitter_method='nls',
    fitter_kwargs={'ndata': 15, 'parallel': False}
)

# Run the experiment
results = exp2d.run()

# Plot the results for the 'phi' parameter from the 'nls_fit' analysis
ax = exp2d.plot(analysis_name='nls_fit', param_to_plot='phi', stat='mean')
ax.set_xlabel(r'Input Modulation Phase, $\psi_{\rm in}$ (rad)')
ax.set_ylabel(r'Input IFO Phase, $\phi_{\rm in}$ (rad)')
ax.figure.get_axes()[1].set_ylabel(r'Fitted IFO Phase, $\hat{\phi}$ (rad)') # Colorbar label
plt.show()